# One-Hot Encoding 

One-hot encoding is one of the most important feature transformation techniques that turns categorical values into binary representations. Machine learning algorithms operate on numerical inputs. Therefore, we have to transform categorical data into some form of numerical representation. One-hot encoding is an excellent candidate for this task because it is easy to understand and straightforward to implement. 

When we apply one-hot encoding to our categorical columns in our dataset, we create a new binary indicator column for every unique value in the original categorical columm. 

Let's consider an example. Let's say we have a feature named `animal` that can have one of three possible values: `Dog`, `Cat` and `Dinosaur`. We would replace the column `animal` with three new columns, one for every potential value of `animal`: `Dog`, `Cat` and `Dinosaur`. Each new column would contain binary values. For example, for every row in which the original column `animal` had the value of `Dog`, the new column `Dog` would have the value of 1. For every row in which the original column `animal` did NOT have the value of `Dog`, the new column `Dog` would have the value of 0. Compare the original dataset below with the resulting dataset after one-hot encoding has been performed on the `animal` column.

<img src='onehotencoding3.png' width=600 height=600 align="left"/>

Recall that to implement a k-Nearest Neighbors model, we cannot have features for which computing the distance is impossible: the string-valued, categorical features. 

In the previous exercise, you removed these values from the dataset before implementing the KNN model. In this demo,  you will see how to use one hot encoding to transform these values instead.

### Import Packages

Let's begin by loading the required packages:

In [1]:
import pandas as pd
import numpy as np
import os 

### Load the Data Set

We will once again work with the "cell2celltrain" data set. This data set is already pre-processed, with the proper formatting, outliers and missing values taken care of, and all numerical columns scaled to the [0, 1] interval.

In [2]:
filename = os.path.join("data", "cell2celltrain.csv")
df = pd.read_csv(filename, header=0)

In [3]:
df.shape

(51047, 58)

In [4]:
df.head()

,CustomerID,Churn,ServiceArea,ChildrenInHH,HandsetRefurbished,HandsetWebCapable,TruckOwner,RVOwner,HomeownershipKnown,BuysViaMailOrder,...,HandsetModels,CurrentEquipmentDays,AgeHH1,AgeHH2,RetentionCalls,RetentionOffersAccepted,ReferralsMadeBySubscriber,IncomeGroup,AdjustmentsToCreditRating,HandsetPrice
0,3000002,True,SEAPOR503,False,False,True,False,False,True,True,...,0.487071,-0.077013,1.387766,-0.883541,4.662897,-0.1283,-0.169283,-0.103411,-0.140707,-0.864858
1,3000010,True,PITHOM412,True,False,False,False,False,True,True,...,-0.616775,3.019920,0.392039,0.871495,-0.180167,-0.1283,-0.169283,0.215243,-0.140707,-0.864858
2,3000014,False,MILMIL414,True,False,False,False,False,False,False,...,-0.616775,3.019920,-0.241605,0.202910,-0.180167,-0.1283,-0.169283,0.533896,-0.140707,-0.368174
3,3000022,False,PITHOM412,False,False,True,False,False,True,True,...,2.694763,0.305179,-0.060564,-0.883541,-0.180167,-0.1283,-0.169283,0.533896,-0.140707,-1.195980
4,3000026,True,OKCTUL918,False,False,False,False,False,True,True,...,1.590917,1.857585,0.663601,1.372934,-0.180167,-0.1283,-0.169283,1.489856,2.469282,-1.195980


### Find the Columns Containing String Values

In [5]:
df.dtypes

CustomerID                     int64
Churn                           bool
ServiceArea                   object
ChildrenInHH                    bool
HandsetRefurbished              bool
HandsetWebCapable               bool
TruckOwner                      bool
RVOwner                         bool
HomeownershipKnown              bool
BuysViaMailOrder                bool
RespondsToMailOffers            bool
OptOutMailings                  bool
NonUSTravel                     bool
OwnsComputer                    bool
HasCreditCard                   bool
NewCellphoneUser                bool
NotNewCellphoneUser             bool
OwnsMotorcycle                  bool
MadeCallToRetentionTeam         bool
CreditRating                  object
PrizmCode                     object
Occupation                    object
Married                       object
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
O

The code cell below finds all columns of type `object`.

In [6]:
to_encode = list(df.select_dtypes(include=['object']).columns)
print(to_encode)


['ServiceArea', 'CreditRating', 'PrizmCode', 'Occupation', 'Married']


Below you will one-hot encode the columns using three different approaches.

## One-Hot Encode the Data Using NumPy

In the last exercise, you removed these columns from DataFrame `df`. This time, we will transform them using One-hot encoding.

There are 5 object-type columns in our DataFrame `df`. Lets inspect the possible number of values each column (feature) may have.

In [7]:
df[to_encode].nunique()

ServiceArea     747
CreditRating      7
PrizmCode         4
Occupation        8
Married           2
dtype: int64

Notice that column `ServiceArea` has 747 potential values. This means we would have to create 747 new binary indicator columns - one column per unique value. That is too many!

Let's handle the special case of column `ServiceArea` first. Transforming this many categorical values would slow down the computation down the line. Instead, we will convert the top 10 most frequent values in column `ServiceArea`. 

In [8]:
top_10_SA = list(df['ServiceArea'].value_counts().head(10).index)

top_10_SA

['NYCBRO917',
 'HOUHOU281',
 'DALDAL214',
 'NYCMAN917',
 'APCFCH703',
 'DALFTW817',
 'SANSAN210',
 'APCSIL301',
 'SANAUS512',
 'SFROAK510']

Now that we have obtained the ten most frequent values for `ServiceArea`, let's transform DataFrame `df` to represent these values numerically.

1. Create new columns to represent `ServiceArea`.

    * Instead of the original `ServiceArea` column, `df` must contain ten one-hot encoded columns: one column for every value in the top 10 most frequent service areas.

    * For example, there will be one column for `NYCBRO917`, one column for `HOUHOU281`, one column for `DALDAL214` and so on. We will name each column 'ServiceArea + '\_' + $<$service area value$>$'. For example, there will be a column named  `ServiceArea_NYCBRO917`.


2. Create values for each column.

    * Each column will have a value of either 0 or 1. 

    * 1 means that the row in question had that corresponding value present in the original `ServiceArea` column. 

    * For example, row 47 in DataFrame `df` originally had the value `DALDAL214` in column `ServiceArea`. After one-hot ending, row 47 will have the value of 1 in new column `ServiceArea_DALDAL214`.
    
The code cell below accomplishes the task of creating ten one-hot encoding columns. 


In [9]:
for value in top_10_SA:
    
    ## Create columns and their values
    df['ServiceArea_'+ value] = np.where(df['ServiceArea']==value,1,0)
    
    
# Remove the original column from your DataFrame df
df.drop(columns = 'ServiceArea', inplace=True)

# Remove from list to_encode
to_encode.remove('ServiceArea')

Inspect DataFrame `df` and see the new columns and their values

In [10]:
df.head()

,CustomerID,Churn,ChildrenInHH,HandsetRefurbished,HandsetWebCapable,TruckOwner,RVOwner,HomeownershipKnown,BuysViaMailOrder,RespondsToMailOffers,...,ServiceArea_NYCBRO917,ServiceArea_HOUHOU281,ServiceArea_DALDAL214,ServiceArea_NYCMAN917,ServiceArea_APCFCH703,ServiceArea_DALFTW817,ServiceArea_SANSAN210,ServiceArea_APCSIL301,ServiceArea_SANAUS512,ServiceArea_SFROAK510
0,3000002,True,False,False,True,False,False,True,True,True,...,0,0,0,0,0,0,0,0,0,0
1,3000010,True,True,False,False,False,False,True,True,True,...,0,0,0,0,0,0,0,0,0,0
2,3000014,False,True,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
3,3000022,False,False,False,True,False,False,True,True,True,...,0,0,0,0,0,0,0,0,0,0
4,3000026,True,False,False,False,False,False,True,True,True,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.columns

Index(['CustomerID', 'Churn', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'HomeownershipKnown',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'OwnsMotorcycle', 'MadeCallToRetentionTeam',
       'CreditRating', 'PrizmCode', 'Occupation', 'Married', 'MonthlyRevenue',
       'MonthlyMinutes', 'TotalRecurringCharge', 'DirectorAssistedCalls',
       'OverageMinutes', 'RoamingCalls', 'PercChangeMinutes',
       'PercChangeRevenues', 'DroppedCalls', 'BlockedCalls', 'UnansweredCalls',
       'CustomerCareCalls', 'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls',
       'InboundCalls', 'PeakCallsInOut', 'OffPeakCallsInOut',
       'DroppedBlockedCalls', 'CallForwardingCalls', 'CallWaitingCalls',
       'MonthsInService', 'UniqueSubs', 'ActiveSubs', 'Handsets',
       'HandsetModels', 'CurrentEquipmentDays', 'AgeHH1', 'AgeH

Let's inspect column `ServiceAreaDALDAL214` in row 47. Remember, it should have a value of 1

In [12]:
df.loc[47]['ServiceArea_DALDAL214']

1

## One-Hot Encode the Data Using Pandas

Now that we have successfully transformed the `ServiceArea` column, let us transform the `Married` column. Let's inspect the values in `Married`.

In [13]:
df['Married']

0        False
1         True
2         True
3        False
4         True
         ...  
51042     True
51043    False
51044    False
51045    False
51046      NaN
Name: Married, Length: 51047, dtype: object

We will perform the same method as above, but using a simpler approach. We will use the Pandas `pd.get_dummies()` function. Recall that we often refer to a binary value that represents a categorial one as "dummy" value or variable.
For more information, consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html). 


In the code cell below, we are specifying which column to encode and the prefix for the new columns. Note that `pd.get_dummies()` returns a new DataFrame with the new one-hot encoded values.

In [14]:
df_Married = pd.get_dummies(df['Married'], prefix='Married_')
df_Married


,Married__False,Married__True
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
51042,0,1
51043,1,0
51044,1,0
51045,1,0


Since the `pd.get_dummies()` function didnt make the changes to the original dataframe `df`, let us add the new DataFrame `df_Married` to DataFrame `df`, and delete the original `Married` column.


In [15]:
# Concatenate with the encoded dataframe:
df = df.join(df_Married)

# Remove the original column from your DataFrame df
df.drop(columns = 'Married', inplace=True)

# Remove from list to_encode
to_encode.remove('Married')


Let's inspect DataFrame `df`

In [16]:
df.columns

Index(['CustomerID', 'Churn', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'HomeownershipKnown',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'OwnsMotorcycle', 'MadeCallToRetentionTeam',
       'CreditRating', 'PrizmCode', 'Occupation', 'MonthlyRevenue',
       'MonthlyMinutes', 'TotalRecurringCharge', 'DirectorAssistedCalls',
       'OverageMinutes', 'RoamingCalls', 'PercChangeMinutes',
       'PercChangeRevenues', 'DroppedCalls', 'BlockedCalls', 'UnansweredCalls',
       'CustomerCareCalls', 'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls',
       'InboundCalls', 'PeakCallsInOut', 'OffPeakCallsInOut',
       'DroppedBlockedCalls', 'CallForwardingCalls', 'CallWaitingCalls',
       'MonthsInService', 'UniqueSubs', 'ActiveSubs', 'Handsets',
       'HandsetModels', 'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2',
      

## One-Hot Encode the Data Using Scikit-Learn

Instead of transforming each column using the NumPy `np.where()` or Pandas `pd.get_dummies()` functions, we can use the more robust `OneHotEncoder` transformation class from `sklearn`. For more information, consult the online [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). Note that you may have to handle with missing values in your data prior to using  `OneHotEncoder`.

In [17]:
from sklearn.preprocessing import OneHotEncoder

# Create the encoder:
encoder = OneHotEncoder(handle_unknown="error", sparse=False)

# Apply the encoder:
df_enc = pd.DataFrame(encoder.fit_transform(df[to_encode]))


# Reistate the original column names:
df_enc.columns = encoder.get_feature_names(to_encode)



Let's glance at the one-hot encoded columns.

In [18]:
df_enc.head()

,CreditRating_1-Highest,CreditRating_2-High,CreditRating_3-Good,CreditRating_4-Medium,CreditRating_5-Low,CreditRating_6-VeryLow,CreditRating_7-Lowest,PrizmCode_Other,PrizmCode_Rural,PrizmCode_Suburban,PrizmCode_Town,Occupation_Clerical,Occupation_Crafts,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


We can now merge the transformed categorical features into DataFrame `df` and remove the original columns that we have just transformed.

In [19]:
# Concatenate with the encoded dataframe:
df = df.join(df_enc)

# Remove the original categorical features from X_train and X_test:
df.drop(columns = to_encode ,axis=1, inplace=True)



Let's now inspect Dataframe `df` and see how it has been transformed. Notice the new dimensions of `df`.

In [20]:
df.columns

Index(['CustomerID', 'Churn', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'HomeownershipKnown',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'OwnsMotorcycle', 'MadeCallToRetentionTeam',
       'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
       'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls',
       'ReceivedCalls', 'OutboundCalls', 'InboundCalls', 'PeakCallsInOut',
       'OffPeakCallsInOut', 'DroppedBlockedCalls', 'CallForwardingCalls',
       'CallWaitingCalls', 'MonthsInService', 'UniqueSubs', 'ActiveSubs',
       'Handsets', 'HandsetModels', 'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2',
       'RetentionCalls', 'RetentionOffersAccepted',
    

In [21]:
df.shape

(51047, 84)